<a href="https://colab.research.google.com/github/NavuruSahithya/dlassignment-2/blob/main/DL_ASSIGNMENT_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#QUESTION1
import requests
import zipfile
import io
import os

# URL for the Dakshina dataset
url = "https://github.com/google-research-datasets/dakshina/archive/refs/heads/master.zip"

# Directory to save the dataset
dataset_dir = "dakshina_dataset"

# Create the directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)

# Download the dataset
response = requests.get(url)

# Check if the download was successful
if response.status_code == 200:
    # Extract the dataset
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(dataset_dir)
    print(f"Dakshina dataset downloaded and extracted to '{dataset_dir}'")
else:
    print(f"Failed to download the dataset: {response.status_code} {response.reason}")


Dakshina dataset downloaded and extracted to 'dakshina_dataset'


In [3]:

from google.colab import files

# Upload your dataset
uploaded = files.upload()


Saving hi.translit.sampled.train.tsv to hi.translit.sampled.train.tsv


In [4]:
import os

# List all files in the current directory (content)
print(os.listdir("/content"))

['.config', 'hi.translit.sampled.train.tsv', 'dakshina_dataset', 'sample_data']


In [10]:
import pandas as pd

# Define the path to the uploaded file
lexicon_path = "/content/hi.translit.sampled.train.tsv"

# Load the file into a DataFrame
df = pd.read_csv(lexicon_path, sep='\t', header=None, names=['native', 'latin', 'attested'])

# Show the first 5 rows
print(df.tail())


          native       latin  attested
44199  ह्वेनसांग  hiuentsang         1
44200  ह्वेनसांग  hsuantsang         1
44201  ह्वेनसांग    hyensang         1
44202  ह्वेनसांग    xuanzang         1
44203          ॐ          om         3


In [6]:
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Parameters
batch_size = 64
epochs = 200
latent_dim = 256
num_samples = 50000  # Limit the number of training samples

# Path to the Dakshina dataset (adjust if you have different path)
data_path = "/content/hi.translit.sampled.train.tsv"

# Step 1: Load the dataset
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    native, latin, attested = line.split("\t")
    input_text = latin.strip()
    target_text = "\t" + native.strip() + "\n"  # Start and end tokens

    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

# Add space character to both input and target characters if not already included
input_characters.add(" ")
target_characters.add(" ")

# Rebuild the input_token_index and target_token_index after adding space
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Step 2: Create empty arrays to store one-hot encodings
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

# Step 3: Fill in the one-hot encoded arrays
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0  # Fix added here
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# Define the sequence-to-sequence model
# Step 4: Define Encoder and Decoder Model
# Encoder model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Step 5: Compile and train the model
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

# Save the model after training
model.save("s2s_model.keras")


Number of samples: 44204
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21
Epoch 1/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.6882 - loss: 1.3214 - val_accuracy: 0.7305 - val_loss: 1.1050
Epoch 2/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7686 - loss: 0.8648 - val_accuracy: 0.7474 - val_loss: 1.0524
Epoch 3/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.7953 - loss: 0.7404 - val_accuracy: 0.7510 - val_loss: 1.0592
Epoch 4/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8145 - loss: 0.6613 - val_accuracy: 0.7551 - val_loss: 1.0926
Epoch 5/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8288 - loss: 0.6016 - val_accuracy: 0.7534 - val_loss: 1.0998
Epoch 6/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8418 - loss: 0.5442 - val_accuracy: 0.7581 - val_loss: 1.0364
Epoch 7/200
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step 

In [7]:
import numpy as np
import keras

# Load the trained model
model = keras.models.load_model("s2s_model.keras")

# Get the encoder model
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

# Get the decoder model
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Function to decode a sequence
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences (simplified for batch size = 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        # Get the output tokens and new states from the decoder model
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        # Sample a token (choose the token with the highest probability)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Example usage:
# Assuming `input_seq` is the one-hot encoded sequence of an input Latin sentence
# decoded_sentence = decode_sequence(input_seq)
# print(decoded_sentence)


In [9]:
import numpy as np

# Define max sequence length and number of tokens (adjust based on your data)
max_encoder_seq_length = 15  # Example maximum length of input sequences
num_encoder_tokens = len(input_token_index)  # Number of unique tokens in the input Latin vocabulary

def encode_input_text(input_text):
    # Create a zeroed-out input array
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")

    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.0
        else:
            print(f"Warning: character '{char}' not in training vocabulary.")

    # Ensure the rest of the input is padded with space token
    for t in range(len(input_text), max_encoder_seq_length):
        encoder_input[0, t, input_token_index[' ']] = 1.0  # Add space for padding

    return encoder_input

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0  # Start token

    # Sampling loop for a batch of sequences (simplified for batch size = 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        # Sample a token (choose the token with the highest probability)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Stop condition: stop if we hit the end token or exceed max length
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence

def translate_latin_to_devanagari():
    print("Word to be translated :")
    #type exit to end user input
    while True:
        user_input = input("Translation word is: ").strip()
        if user_input.lower() == "exit":
            break

        input_seq = encode_input_text(user_input)
        decoded_sentence = decode_sequence(input_seq)
        print("Devanagari transliteration:", decoded_sentence.strip())

# Call the function to start interactive translation
translate_latin_to_devanagari()


Word to be translated :
Translation word is: ram
Devanagari transliteration: राम
Translation word is: krishna
Devanagari transliteration: कृष्णन
Translation word is: rakesh
Devanagari transliteration: राकेश
Translation word is: raj
Devanagari transliteration: रज
Translation word is: lakshmi
Devanagari transliteration: लाक्ष्मि
Translation word is: sai
Devanagari transliteration: साईया
Translation word is: exit
